# Chapter 1: MCMC基礎理論

## 学習目標
- MCMCの基本概念と必要性を理解する
- マルコフ連鎖の基本性質を学ぶ
- 定常分布と詳細釣り合い条件を理解する
- 確率分布からのサンプリング問題を把握する

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.integrate import quad
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
np.random.seed(42)

## 1.1 MCMCとは何か？

**MCMC（Markov Chain Monte Carlo）**は、複雑な確率分布からサンプリングを行うための強力な手法です。

### なぜMCMCが必要なのか？

以下のような状況で直接サンプリングが困難な場合があります：
- 正規化定数（分配関数）の計算が困難
- 高次元空間での積分計算
- 複雑な形状の確率分布
- ベイズ推論における事後分布

### 例：正規化定数が不明な分布

以下のような分布を考えてみましょう：
$$p(x) = \frac{1}{Z} \exp\left(-\frac{x^4}{4} + \frac{x^2}{2}\right)$$

ここで$Z$は正規化定数ですが、これを解析的に計算するのは困難です。

In [ ]:
def unnormalized_pdf(x):
    """正規化されていない確率密度関数"""
    return np.exp(-x**4/4 + x**2/2)

# 数値積分で正規化定数を近似計算
Z, _ = quad(unnormalized_pdf, -10, 10)
print(f"正規化定数 Z ≈ {Z:.4f}")

def normalized_pdf(x):
    return unnormalized_pdf(x) / Z

# 可視化
x = np.linspace(-3, 3, 1000)
y_unnorm = unnormalized_pdf(x)
y_norm = normalized_pdf(x)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(x, y_unnorm, 'b-', linewidth=2)
ax1.set_title('Unnormalized Distribution')
ax1.set_xlabel('x')
ax1.set_ylabel('f(x)')

ax2.plot(x, y_norm, 'r-', linewidth=2)
ax2.set_title('Normalized Distribution')
ax2.set_xlabel('x')
ax2.set_ylabel('p(x)')

plt.tight_layout()
plt.show()

## 1.2 マルコフ連鎖の基本

**マルコフ連鎖**は、MCMCの核となる概念です。

### マルコフ性
現在の状態のみに依存して次の状態が決まる性質：
$$P(X_{t+1} | X_t, X_{t-1}, ..., X_0) = P(X_{t+1} | X_t)$$

### 遷移確率行列
状態$i$から状態$j$への遷移確率を$p_{ij}$とすると：
$$P = \begin{pmatrix}
p_{11} & p_{12} & \cdots & p_{1n} \\
p_{21} & p_{22} & \cdots & p_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
p_{n1} & p_{n2} & \cdots & p_{nn}
\end{pmatrix}$$

### 例：簡単なマルコフ連鎖のシミュレーション

In [ ]:
def simulate_markov_chain(P, initial_state, n_steps):
    """
    マルコフ連鎖のシミュレーション
    
    Parameters:
    - P: 遷移確率行列
    - initial_state: 初期状態
    - n_steps: ステップ数
    """
    n_states = P.shape[0]
    states = np.zeros(n_steps, dtype=int)
    states[0] = initial_state
    
    for t in range(1, n_steps):
        current_state = states[t-1]
        # 現在の状態から次の状態への遷移確率に基づいてサンプリング
        states[t] = np.random.choice(n_states, p=P[current_state])
    
    return states

# 3状態のマルコフ連鎖の例
P = np.array([
    [0.7, 0.2, 0.1],  # 状態0からの遷移確率
    [0.3, 0.4, 0.3],  # 状態1からの遷移確率
    [0.2, 0.3, 0.5]   # 状態2からの遷移確率
])

# シミュレーション実行
chain = simulate_markov_chain(P, initial_state=0, n_steps=1000)

# 結果の可視化
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# 状態の時系列
ax1.plot(chain[:200], 'o-', markersize=3, linewidth=1)
ax1.set_title('Markov Chain State Transitions (first 200 steps)')
ax1.set_xlabel('Time Step')
ax1.set_ylabel('State')
ax1.set_yticks([0, 1, 2])
ax1.grid(True, alpha=0.3)

# 状態の分布
state_counts = np.bincount(chain)
state_probs = state_counts / len(chain)

ax2.bar(range(3), state_probs, alpha=0.7, color=['blue', 'orange', 'green'])
ax2.set_title('Empirical State Distribution')
ax2.set_xlabel('State')
ax2.set_ylabel('Probability')
ax2.set_xticks([0, 1, 2])

plt.tight_layout()
plt.show()

print(f"経験的状態分布: {state_probs}")

## 1.3 定常分布

**定常分布**$\pi$は、以下の条件を満たす分布です：
$$\pi = \pi P$$

これは、分布$\pi$がマルコフ連鎖の長期的な振る舞いを表すことを意味します。

In [ ]:
def find_stationary_distribution(P, method='eigenvalue'):
    """
    定常分布の計算
    
    Parameters:
    - P: 遷移確率行列
    - method: 'eigenvalue' または 'power_iteration'
    """
    if method == 'eigenvalue':
        # 固有値・固有ベクトル法
        eigenvals, eigenvecs = np.linalg.eig(P.T)
        # 固有値1に対応する固有ベクトルを探す
        idx = np.argmin(np.abs(eigenvals - 1.0))
        stationary = np.real(eigenvecs[:, idx])
        stationary = stationary / np.sum(stationary)  # 正規化
        return stationary
    
    elif method == 'power_iteration':
        # べき乗法による近似
        pi = np.ones(P.shape[0]) / P.shape[0]  # 初期分布
        for _ in range(1000):
            pi_new = pi @ P
            if np.allclose(pi, pi_new, atol=1e-10):
                break
            pi = pi_new
        return pi

# 先ほどの遷移行列Pの定常分布を計算
stationary_exact = find_stationary_distribution(P, method='eigenvalue')
stationary_approx = find_stationary_distribution(P, method='power_iteration')

print("理論的定常分布 (固有値法):")
print(f"π = [{stationary_exact[0]:.4f}, {stationary_exact[1]:.4f}, {stationary_exact[2]:.4f}]")

print("\n近似定常分布 (べき乗法):")
print(f"π = [{stationary_approx[0]:.4f}, {stationary_approx[1]:.4f}, {stationary_approx[2]:.4f}]")

print(f"\n経験分布との比較:")
print(f"経験分布: [{state_probs[0]:.4f}, {state_probs[1]:.4f}, {state_probs[2]:.4f}]")
print(f"理論値:   [{stationary_exact[0]:.4f}, {stationary_exact[1]:.4f}, {stationary_exact[2]:.4f}]")

## 1.4 詳細釣り合い条件

**詳細釣り合い条件**（Detailed Balance Condition）は、MCMCアルゴリズムの設計において重要な概念です：

$$\pi(x) P(x \rightarrow y) = \pi(y) P(y \rightarrow x)$$

この条件は、定常状態において各状態間のフローが釣り合っていることを意味します。

In [ ]:
def check_detailed_balance(P, pi):
    """
    詳細釣り合い条件のチェック
    """
    n = P.shape[0]
    detailed_balance_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            # π(i) * P(i→j) と π(j) * P(j→i) を比較
            left_side = pi[i] * P[i, j]
            right_side = pi[j] * P[j, i]
            detailed_balance_matrix[i, j] = left_side - right_side
    
    return detailed_balance_matrix

# 詳細釣り合い条件をチェック
db_matrix = check_detailed_balance(P, stationary_exact)

print("詳細釣り合い条件のチェック:")
print("π(i)P(i→j) - π(j)P(j→i) の値:")
print(db_matrix)
print(f"\n最大絶対誤差: {np.max(np.abs(db_matrix)):.6f}")

# この遷移行列は詳細釣り合い条件を満たさない（一般的な場合）
print("\n注意: 一般的なマルコフ連鎖では詳細釣り合い条件は成立しません。")
print("MCMCアルゴリズムでは、この条件を満たすように遷移確率を設計します。")

## 1.5 MCMCの基本的なアイデア

MCMCの基本的なアイデアは以下の通りです：

1. **目標分布**：サンプリングしたい分布$\pi(x)$
2. **マルコフ連鎖の構築**：$\pi(x)$を定常分布とするマルコフ連鎖を構築
3. **サンプリング**：十分長い時間経過後のサンプルは$\pi(x)$に従う

### 重要な性質
- **エルゴード性**：任意の初期状態から定常分布に収束
- **非可約性**：すべての状態間が到達可能
- **非周期性**：周期的な振動がない

## 1.6 演習問題

### 問題1：マルコフ連鎖のシミュレーション
以下の遷移確率行列を持つマルコフ連鎖をシミュレートし、定常分布を求めなさい。

In [ ]:
# 演習用の遷移行列
P_exercise = np.array([
    [0.5, 0.3, 0.2],
    [0.2, 0.6, 0.2],
    [0.1, 0.4, 0.5]
])

# ここにコードを書いてください
# 1. マルコフ連鎖をシミュレート
# 2. 定常分布を理論的に計算
# 3. 経験分布と比較

# 解答例（コメントアウト）
# chain_ex = simulate_markov_chain(P_exercise, 0, 10000)
# stationary_ex = find_stationary_distribution(P_exercise)
# empirical_ex = np.bincount(chain_ex) / len(chain_ex)
# print(f"理論値: {stationary_ex}")
# print(f"経験値: {empirical_ex}")

### 問題2：収束の可視化
異なる初期状態から始めたマルコフ連鎖が定常分布に収束する様子を可視化しなさい。

In [ ]:
# ここにコードを書いてください
# 1. 複数の初期状態からマルコフ連鎖を開始
# 2. 各状態の確率の時間変化をプロット
# 3. 定常分布への収束を観察

pass  # 学習者が実装

## まとめ

この章では、MCMCの基礎となる以下の概念を学びました：

1. **MCMCの必要性**：複雑な分布からのサンプリング問題
2. **マルコフ連鎖**：マルコフ性と遷移確率行列
3. **定常分布**：長期的な振る舞いと計算方法
4. **詳細釣り合い条件**：MCMCアルゴリズム設計の原理

次の章では、これらの理論を基に具体的なMCMCアルゴリズムであるメトロポリス・ヘイスティングス法を学習します。

### 参考文献
- Robert, C. & Casella, G. (2004). Monte Carlo Statistical Methods
- Brooks, S. et al. (2011). Handbook of Markov Chain Monte Carlo